## Find Missing Routes: 2 operators. 
* [Issue](https://github.com/cal-itp/data-analyses/issues/1312): Capital Corridor doesn't have any rail routes. 
* [Most of Santa Maria's routes not showing up in GTFS Digest](https://github.com/cal-itp/data-analyses/issues/1313)
* `cd data-analyses/rt_segment_speeds && pip install -r requirements.txt && cd ../_shared_utils && make setup_env && cd ../gtfs_digest`
* 1/7: the routes are appearing in `the longest shape` but not appearing in `route_typologies`

In [164]:
import geopandas as gpd
import merge_data
import numpy as np
import pandas as pd
from segment_speed_utils import (
    gtfs_schedule_wrangling,
    helpers,
    metrics,
    segment_calcs,
    time_series_utils,
)
from shared_utils import (
    catalog_utils,
    portfolio_utils,
    rt_dates,
    rt_utils,
    time_helpers,
)
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
org_name_lists = ["Capitol Corridor Joint Powers Authority", "City of Santa Maria"]

In [4]:
analysis_date_list = rt_dates.y2024_dates

In [5]:
one_analysis_date = "2024-11-13"

In [6]:
schd_keys = [
    "5a8721fe96786fcd25fba1f8a0ee6358",
    "73105f2d1cabc8170ab066d96863c5d5",
    "f5a749dd65924e025b1293c58f95f8d6",
]

In [7]:
import sys

sys.path.append("../gtfs_funnel/")
import operator_scheduled_stats
import schedule_stats_by_route_direction

In [62]:
def preview(df):
    df2 = df[
        ["schedule_gtfs_dataset_key", "route_id", "direction_id"]
    ].drop_duplicates()
    display(df2)

### Fix `schd_vp_url`

In [8]:
schd_vp_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.route_schedule_vp}.parquet"

In [15]:
schd_vp_df = pd.read_parquet(schd_vp_url)

In [16]:
schd_vp_df2 = schd_vp_df.loc[schd_vp_df.organization_name.isin(org_name_lists)]

In [17]:
schd_vp_df2.route_id.unique()

array(['b3848f93-d26b-48a9-b6a6-5de22a4eab47', '5', 'Shuttle', 'CC'],
      dtype=object)

###  Check out `rt_segment_speeds/segment_speed_utils/gtfs_schedule_wrangling`
* https://github.com/cal-itp/data-analyses/blob/4dc340343a60b45ad94217c3efd91f807b03ebc2/rt_segment_speeds/segment_speed_utils/gtfs_schedule_wrangling.py 
* Tiffany: <i>Can you try specifying the dropna argument inside pandas groupby? Our pandas version has gone through upgrades, from 0.25 to now 1.5 ), and this argument was introduced in 1.1 and since it's dropna=True, that's probably what's driving the the row behavior.</i>
* It worked! Now time to rerun stuff further down the pipeline and see what happens.

In [20]:
def most_common_shape_by_route_direction(analysis_date: str) -> gpd.GeoDataFrame:
    """
    Find shape_id with most trips for that route-direction.
    Merge in shape geometry.
    """
    route_dir_cols = ["gtfs_dataset_key", "route_id", "direction_id"]

    keep_trip_cols = route_dir_cols + [
        "trip_instance_key",
        "shape_id",
        "shape_array_key",
    ]

    trips = helpers.import_scheduled_trips(
        analysis_date, columns=keep_trip_cols, get_pandas=True
    ).rename(columns={"schedule_gtfs_dataset_key": "gtfs_dataset_key"})
    sorting_order = [True for i in route_dir_cols]

    most_common_shape = (
        trips.groupby(
            route_dir_cols + ["shape_id", "shape_array_key"],
            observed=True,
            group_keys=False,
            dropna=False,
        )
        .agg({"trip_instance_key": "count"})
        .reset_index()
        .sort_values(
            route_dir_cols + ["trip_instance_key"], ascending=sorting_order + [False]
        )
        .drop_duplicates(subset=route_dir_cols)
        .reset_index(drop=True)[route_dir_cols + ["shape_id", "shape_array_key"]]
    ).rename(
        columns={
            "gtfs_dataset_key": "schedule_gtfs_dataset_key",
            "shape_id": "common_shape_id",
        }
    )

    shape_geom = helpers.import_scheduled_shapes(
        analysis_date,
        columns=["shape_array_key", "geometry"],
    )

    common_shape_geom = pd.merge(
        shape_geom, most_common_shape, on="shape_array_key", how="inner"
    ).drop(columns="shape_array_key")

    route_info = (
        helpers.import_scheduled_trips(
            analysis_date,
            columns=[
                "gtfs_dataset_key",
                "route_id",
                "route_long_name",
                "route_short_name",
                "route_desc",
            ],
        )
        .drop_duplicates()
        .pipe(portfolio_utils.add_route_name)
        .drop(columns=["route_long_name", "route_short_name", "route_desc"])
    )

    del shape_geom, most_common_shape

    common_shape_geom2 = pd.merge(
        common_shape_geom,
        route_info.rename(columns={"route_name_used": "route_name"}),
        on=["schedule_gtfs_dataset_key", "route_id"],
    )

    return common_shape_geom2

In [23]:
common_shape_test = most_common_shape_by_route_direction(one_analysis_date)

In [26]:
common_shape_test.columns

Index(['geometry', 'schedule_gtfs_dataset_key', 'route_id', 'direction_id',
       'common_shape_id', 'route_name'],
      dtype='object')

In [27]:
common_shape_test2 = common_shape_test.loc[
    common_shape_test.schedule_gtfs_dataset_key.isin(schd_keys)
]

In [28]:
common_shape_test2.route_id.unique()

array(['7', '6', 'CC', '8', 'Mall', '12X', '13X', '11', '30', 'Shuttle',
       '8a7c42f9-51e4-4848-bf88-30c210f149ad', '2', '3', '1B', '20', 'SF',
       '5', '4', '9', '1'], dtype=object)

In [31]:
[rt_dates.DATES["dec2024"]] + [rt_dates.DATES["nov2024"]]

['2024-12-11', '2024-11-13']

In [32]:
GTFS_DATA_DICT.rt_vs_schedule_tables.sched_route_direction_metrics

'schedule_route_dir/schedule_route_direction_metrics'

### Breakdown `gtfs_digest/merge_data.`

#### Line 294: `df_sched` is already missing a lot of the routes.

In [9]:
# Get cardinal direction for each route
df_sched = merge_data.concatenate_schedule_by_route_direction(analysis_date_list)

In [10]:
df_sched2 = df_sched.loc[df_sched.schedule_gtfs_dataset_key.isin(schd_keys)]

In [11]:
df_sched2.route_id.value_counts()

Shuttle                                 72
5                                       27
b3848f93-d26b-48a9-b6a6-5de22a4eab47     9
Name: route_id, dtype: int64

##### Go back to `gtfs_funnel/schedule_stats_by_route_direction`
* https://github.com/cal-itp/data-analyses/blob/1ba0f544a01f99966a6e210dd11666b4fe4a146e/gtfs_funnel/schedule_stats_by_route_direction.py#L190
* Test 1: Updated `gtfs_schedule_wrangling` but a lot of routes are still missing. 

In [33]:
RT_SCHED_GCS

'gs://calitp-analytics-data/data-analyses/rt_vs_schedule/'

In [12]:
GTFS_DATA_DICT.rt_vs_schedule_tables.sched_route_direction_metrics

'schedule_route_dir/schedule_route_direction_metrics'

In [37]:
nov_rt_sched = pd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/rt_vs_schedule/AH_schedule_route_dir/schedule_route_direction_metrics_2024-12-11.parquet"
)

In [38]:
nov_rt_sched2 = nov_rt_sched.loc[nov_rt_sched.schedule_gtfs_dataset_key.isin(schd_keys)]

In [39]:
nov_rt_sched2.route_id.unique()

array(['Shuttle', '5'], dtype=object)

##### `assemble_scheduled_trip_metrics`: nothing is missing

In [13]:
trip_metrics = schedule_stats_by_route_direction.assemble_scheduled_trip_metrics(
    one_analysis_date, GTFS_DATA_DICT
)

In [14]:
trip_metrics.head(1)

,schedule_gtfs_dataset_key,trip_instance_key,median_stop_meters,time_of_day,scheduled_service_minutes,route_id,direction_id
0,0139b1253130b33adcd4b3a4490530d2,014dd8051849e5252df704ca9c381fd9,559.44,PM Peak,23.00,D4,NaN


In [18]:
trip_metrics2 = trip_metrics.loc[trip_metrics.schedule_gtfs_dataset_key.isin(schd_keys)]

In [63]:
preview(trip_metrics2)

,schedule_gtfs_dataset_key,route_id,direction_id
49724,73105f2d1cabc8170ab066d96863c5d5,30,NaN
49725,73105f2d1cabc8170ab066d96863c5d5,3,NaN
49727,73105f2d1cabc8170ab066d96863c5d5,20,NaN
49728,73105f2d1cabc8170ab066d96863c5d5,4,NaN
49729,73105f2d1cabc8170ab066d96863c5d5,Mall,NaN
49730,73105f2d1cabc8170ab066d96863c5d5,5,0.00
49731,73105f2d1cabc8170ab066d96863c5d5,11,NaN
49732,73105f2d1cabc8170ab066d96863c5d5,7,NaN
49733,73105f2d1cabc8170ab066d96863c5d5,9,NaN
49735,73105f2d1cabc8170ab066d96863c5d5,1,NaN


##### `schedule_metrics_by_route_direction` 
* Something is causing routes to drop off in  even though I took out `direction_id`
* Break it out even more.

In [40]:
route_group_merge_cols = ["schedule_gtfs_dataset_key", "route_id", "direction_id"]

In [41]:
route_group_merge_cols_no_dir_id = [
    "schedule_gtfs_dataset_key",
    "route_id",
]

In [42]:
route_dir_metrics = (
    schedule_stats_by_route_direction.schedule_metrics_by_route_direction(
        trip_metrics2, one_analysis_date, route_group_merge_cols
    )
)

In [43]:
route_dir_metrics[
    ["schedule_gtfs_dataset_key", "route_id", "direction_id"]
].drop_duplicates()

,schedule_gtfs_dataset_key,route_id,direction_id
0,f5a749dd65924e025b1293c58f95f8d6,Shuttle,1.00
3,f5a749dd65924e025b1293c58f95f8d6,Shuttle,0.00
6,73105f2d1cabc8170ab066d96863c5d5,5,0.00


In [44]:
route_dir_metrics_wo_dir_id = (
    schedule_stats_by_route_direction.schedule_metrics_by_route_direction(
        trip_metrics2, one_analysis_date, route_group_merge_cols_no_dir_id
    )
)

In [45]:
route_dir_metrics_wo_dir_id[
    ["schedule_gtfs_dataset_key", "route_id", "direction_id"]
].drop_duplicates()

,schedule_gtfs_dataset_key,route_id,direction_id
0,f5a749dd65924e025b1293c58f95f8d6,Shuttle,1.00
3,f5a749dd65924e025b1293c58f95f8d6,Shuttle,0.00
6,73105f2d1cabc8170ab066d96863c5d5,5,0.00


In [46]:
route_group_merge_cols = ["schedule_gtfs_dataset_key", "route_id", "direction_id"]

##### `gtfs_schedule_wrangling.aggregate_time_of_day_to_peak_offpeak` is missing a lot of routes -> break it out.

In [48]:
service_freq_df = gtfs_schedule_wrangling.aggregate_time_of_day_to_peak_offpeak(
    trip_metrics2, route_group_merge_cols, long_or_wide="long"
)

In [49]:
service_freq_df.head(2)

,schedule_gtfs_dataset_key,route_id,direction_id,n_trips,time_period,peak_offpeak,frequency
0,73105f2d1cabc8170ab066d96863c5d5,5,0.00,18,all_day,NaN,0.75
1,f5a749dd65924e025b1293c58f95f8d6,CC,0.00,12,all_day,NaN,0.50


In [50]:
service_freq_df.route_id.unique()

array(['5', 'CC', 'SF', 'Shuttle'], dtype=object)

##### Changed `count_trips_by_group` to have argument `dropna=False` in `groupby`

In [51]:
def count_trips_by_group(df: pd.DataFrame, group_cols: list):
    """
    Given a df with trip_instance_key and an arbitrary list of
    group_cols, return trip counts by group.
    """
    assert "trip_instance_key" in df.columns
    df = (
        df.groupby(group_cols, dropna=False)
        .agg({"trip_instance_key": "count"})
        .reset_index()
    )
    df = df.rename(columns={"trip_instance_key": "n_trips"})
    return df

In [58]:
def add_peak_offpeak_column(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add a single peak_offpeak column based on the time-of-day column.
    """
    df = df.assign(peak_offpeak=df.time_of_day.map(time_helpers.TIME_OF_DAY_DICT))

    return df

In [54]:
def aggregate_time_of_day_to_peak_offpeak(
    df: pd.DataFrame,
    group_cols: list,
    long_or_wide: str,
) -> pd.DataFrame:
    """
    Aggregate time-of-day bins into peak/offpeak periods.
    Return n_trips and frequency for grouping of columns (route-direction, etc).
    Allow wide or long df to be returned.
    """
    peak_hours = sum(
        v
        for k, v in time_helpers.HOURS_BY_TIME_OF_DAY.items()
        if k in time_helpers.PEAK_PERIODS
    )

    offpeak_hours = sum(
        v
        for k, v in time_helpers.HOURS_BY_TIME_OF_DAY.items()
        if k not in time_helpers.PEAK_PERIODS
    )

    df = add_peak_offpeak_column(df)

    all_day = count_trips_by_group(df, group_cols).assign(time_period="all_day")
    peak_offpeak = count_trips_by_group(df, group_cols + ["peak_offpeak"]).rename(
        {"peak_offpeak": "time_period"}
    )

    df2 = pd.concat([all_day, peak_offpeak], axis=0, ignore_index=True)

    # Add service frequency (trips per hour)
    # there are different number of hours in peak and offpeak periods
    df2 = df2.assign(
        frequency=df2.apply(
            lambda x: round(x.n_trips / peak_hours, 2)
            if x.time_period == "peak"
            else round(x.n_trips / offpeak_hours, 2)
            if x.time_period == "offpeak"
            else round(x.n_trips / (peak_hours + offpeak_hours), 2),
            axis=1,
        )
    )

    if long_or_wide == "long":
        return df2

    elif long_or_wide == "wide":
        # Reshape from wide to long
        # get rid of multiindex column names
        df3 = df2.pivot(
            index=group_cols, columns="time_period", values=["n_trips", "frequency"]
        )

        df3.columns = [f"{b}_{a}" for a, b in df3.columns]
        df3 = df3.reset_index()

        return df3

In [59]:
service_freq_df_test1 = aggregate_time_of_day_to_peak_offpeak(
    trip_metrics2, route_group_merge_cols, long_or_wide="long"
)

In [64]:
preview(service_freq_df_test1)

,schedule_gtfs_dataset_key,route_id,direction_id
0,73105f2d1cabc8170ab066d96863c5d5,1,NaN
1,73105f2d1cabc8170ab066d96863c5d5,11,NaN
2,73105f2d1cabc8170ab066d96863c5d5,12X,NaN
3,73105f2d1cabc8170ab066d96863c5d5,13X,NaN
4,73105f2d1cabc8170ab066d96863c5d5,1B,NaN
5,73105f2d1cabc8170ab066d96863c5d5,2,NaN
6,73105f2d1cabc8170ab066d96863c5d5,20,NaN
7,73105f2d1cabc8170ab066d96863c5d5,3,NaN
8,73105f2d1cabc8170ab066d96863c5d5,30,NaN
9,73105f2d1cabc8170ab066d96863c5d5,4,NaN


In [65]:
service_freq_df_test1.head(2)

,schedule_gtfs_dataset_key,route_id,direction_id,n_trips,time_period,peak_offpeak,frequency
0,73105f2d1cabc8170ab066d96863c5d5,1,NaN,19,all_day,NaN,0.79
1,73105f2d1cabc8170ab066d96863c5d5,11,NaN,22,all_day,NaN,0.92


In [66]:
service_freq_df_test2 = aggregate_time_of_day_to_peak_offpeak(
    trip_metrics, route_group_merge_cols, long_or_wide="long"
)

In [68]:
service_freq_df_test2.route_id.nunique()

1825

In [70]:
service_freq_df_test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13113 entries, 0 to 13112
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   schedule_gtfs_dataset_key  13113 non-null  object 
 1   route_id                   13113 non-null  object 
 2   direction_id               12610 non-null  float64
 3   n_trips                    13113 non-null  int64  
 4   time_period                4754 non-null   object 
 5   peak_offpeak               8359 non-null   object 
 6   frequency                  13113 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 717.2+ KB


##### `metrics_df` portion of `gtfs_funnel.schedule_stas_by_route_direction.schedule_metrics_by_route_direction`
* Did `dropna=False` to get all the routes. 
* Without `dropna=False`, all the routes disappear.

In [112]:
route_dir_metrics = (
    trip_metrics2.groupby(
        route_group_merge_cols, observed=True, group_keys=False, dropna=False
    )
    .agg(
        {
            "median_stop_meters": "mean",
            # take mean of the median stop spacing for trip
            # does this make sense?
            # median is the single boiled down metric at the trip-level
            "scheduled_service_minutes": "mean",
        }
    )
    .reset_index()
    .rename(
        columns={
            "median_stop_meters": "avg_stop_meters",
            "scheduled_service_minutes": "avg_scheduled_service_minutes",
        }
    )
)

In [73]:
preview(metrics_df)

,schedule_gtfs_dataset_key,route_id,direction_id
0,73105f2d1cabc8170ab066d96863c5d5,1,NaN
1,73105f2d1cabc8170ab066d96863c5d5,11,NaN
2,73105f2d1cabc8170ab066d96863c5d5,12X,NaN
3,73105f2d1cabc8170ab066d96863c5d5,13X,NaN
4,73105f2d1cabc8170ab066d96863c5d5,1B,NaN
5,73105f2d1cabc8170ab066d96863c5d5,2,NaN
6,73105f2d1cabc8170ab066d96863c5d5,20,NaN
7,73105f2d1cabc8170ab066d96863c5d5,3,NaN
8,73105f2d1cabc8170ab066d96863c5d5,30,NaN
9,73105f2d1cabc8170ab066d96863c5d5,4,NaN


In [74]:
metrics_df2 = (
    trip_metrics2.groupby(route_group_merge_cols, observed=True, group_keys=False)
    .agg(
        {
            "median_stop_meters": "mean",
            # take mean of the median stop spacing for trip
            # does this make sense?
            # median is the single boiled down metric at the trip-level
            "scheduled_service_minutes": "mean",
        }
    )
    .reset_index()
    .rename(
        columns={
            "median_stop_meters": "avg_stop_meters",
            "scheduled_service_minutes": "avg_scheduled_service_minutes",
        }
    )
)

In [75]:
preview(metrics_df2)

,schedule_gtfs_dataset_key,route_id,direction_id
0,73105f2d1cabc8170ab066d96863c5d5,5,0.00
1,f5a749dd65924e025b1293c58f95f8d6,CC,0.00
2,f5a749dd65924e025b1293c58f95f8d6,CC,1.00
3,f5a749dd65924e025b1293c58f95f8d6,SF,0.00
4,f5a749dd65924e025b1293c58f95f8d6,SF,1.00
5,f5a749dd65924e025b1293c58f95f8d6,Shuttle,0.00
6,f5a749dd65924e025b1293c58f95f8d6,Shuttle,1.00


In [77]:
ROUTE_TYPOLOGIES = GTFS_DATA_DICT.schedule_tables.route_typologies

In [79]:
route_typologies = pd.read_parquet(
    f"{SCHED_GCS}{ROUTE_TYPOLOGIES}_{one_analysis_date}.parquet",
    columns=route_group_merge_cols
    + [
        "is_coverage",
        "is_downtown_local",
        "is_local",
        "is_rapid",
        "is_express",
        "is_rail",
    ],
)

In [81]:
cardinal_dir_df = (
    schedule_stats_by_route_direction.cardinal_direction_for_route_direction(
        one_analysis_date, GTFS_DATA_DICT
    )
)

##### `cardinal_dir_df` also gets rid of a lot of stuff.

In [84]:
cardinal_dir_df2 = cardinal_dir_df.loc[
    cardinal_dir_df.schedule_gtfs_dataset_key.isin(schd_keys)
]

In [85]:
preview(cardinal_dir_df2)

,schedule_gtfs_dataset_key,route_id,direction_id
2581,73105f2d1cabc8170ab066d96863c5d5,5,0.00
3947,f5a749dd65924e025b1293c58f95f8d6,CC,0.00
3948,f5a749dd65924e025b1293c58f95f8d6,CC,1.00
4225,f5a749dd65924e025b1293c58f95f8d6,SF,0.00
4226,f5a749dd65924e025b1293c58f95f8d6,SF,1.00
4254,f5a749dd65924e025b1293c58f95f8d6,Shuttle,0.00
4255,f5a749dd65924e025b1293c58f95f8d6,Shuttle,1.00


In [88]:
STOP_TIMES_FILE = GTFS_DATA_DICT.rt_vs_schedule_tables.stop_times_direction

In [89]:
STOP_TIMES_FILE

'stop_times_direction'

In [91]:
stop_times_df = pd.read_parquet(
    f"{RT_SCHED_GCS}{STOP_TIMES_FILE}_{one_analysis_date}.parquet",
    filters=[[("stop_primary_direction", "!=", "Unknown")]],
)

In [93]:
stop_times_df.head(1)

,feed_key,stop_id,stop_sequence,schedule_gtfs_dataset_key,trip_instance_key,shape_array_key,stop_name,geometry,prior_stop_sequence,subseq_stop_sequence,stop_pair,stop_pair_name,stop_primary_direction,stop_meters
0,3b357b65167b3749dff376c9d3624f09,c24c75f6-8397-4d8c-a4e5-72444083a9c8,2,c4726e0acfbcbd26e1dc38b8bd046c03,000000c923cd55be781113b823ce9879,a8ebcf8a2e79025380807e9d58b26f52,Toy Story Bus Stop,b'\x01\x01\x00\x00\x003\xe2\x8a}\r\x8a\x07A\x98H\x01\xc87e\x1c\xc1',1,<NA>,c24c75f6-8397-4d8c-a4e5-72444083a9c8__,Toy Story Bus Stop__,Northbound,1127.76


In [94]:
stop_times_df2 = stop_times_df.loc[
    stop_times_df.schedule_gtfs_dataset_key.isin(schd_keys)
]

In [96]:
stop_times_df2.head(2)

,feed_key,stop_id,stop_sequence,schedule_gtfs_dataset_key,trip_instance_key,shape_array_key,stop_name,geometry,prior_stop_sequence,subseq_stop_sequence,stop_pair,stop_pair_name,stop_primary_direction,stop_meters
6324,c86a471a1a4c36bb8cf7da9d6a20f202,f09af637-87de-4bdb-bf49-660539686c97,2,73105f2d1cabc8170ab066d96863c5d5,005bb393ed8b22ca4d8e7cc8d7895231,c6e9cda0db8bf76bc535f590ca1fccb5,Broadway at Stowell,b'\x01\x01\x00\x00\x00\x8b\x1a\xbc\x0f\x88q\xe3\xc0(q\xf2\xcfo\xe3\x14\xc1',1,3,f09af637-87de-4bdb-bf49-660539686c97__47def414-f158-496a-91cb-5f3fb0aa406c,Broadway at Stowell__Betteravia at Miller (Panda Express),Southbound,1275.56
6325,c86a471a1a4c36bb8cf7da9d6a20f202,47def414-f158-496a-91cb-5f3fb0aa406c,3,73105f2d1cabc8170ab066d96863c5d5,005bb393ed8b22ca4d8e7cc8d7895231,c6e9cda0db8bf76bc535f590ca1fccb5,Betteravia at Miller (Panda Express),"b'\x01\x01\x00\x00\x00.\xacHd""Q\xe3\xc0x\x83P\x05\\\xfc\x14\xc1'",2,4,47def414-f158-496a-91cb-5f3fb0aa406c__a94160c1-bd99-4898-921f-941aa748ce6f,Betteravia at Miller (Panda Express)__McCoy at Broadway (Outbound),Southbound,1615.97


In [97]:
trip_scheduled_col = [
    "route_id",
    "trip_instance_key",
    "gtfs_dataset_key",
    "shape_array_key",
    "direction_id",
    "route_long_name",
    "route_short_name",
    "route_desc",
    "name",
]

trips_df = helpers.import_scheduled_trips(
    one_analysis_date, columns=trip_scheduled_col, get_pandas=True
)

In [99]:
merge_cols = ["trip_instance_key", "schedule_gtfs_dataset_key", "shape_array_key"]

In [100]:
stop_times_with_trip = pd.merge(stop_times_df2, trips_df, on=merge_cols)

In [101]:
preview(stop_times_with_trip)

,schedule_gtfs_dataset_key,route_id,direction_id
0,73105f2d1cabc8170ab066d96863c5d5,30,NaN
26,73105f2d1cabc8170ab066d96863c5d5,3,NaN
66,73105f2d1cabc8170ab066d96863c5d5,20,NaN
75,f5a749dd65924e025b1293c58f95f8d6,Shuttle,0.00
79,73105f2d1cabc8170ab066d96863c5d5,4,NaN
104,f5a749dd65924e025b1293c58f95f8d6,SF,1.00
105,73105f2d1cabc8170ab066d96863c5d5,Mall,NaN
111,73105f2d1cabc8170ab066d96863c5d5,5,0.00
147,73105f2d1cabc8170ab066d96863c5d5,11,NaN
168,73105f2d1cabc8170ab066d96863c5d5,7,NaN


In [102]:
main_cols = ["route_id", "schedule_gtfs_dataset_key", "direction_id"]

##### Changing dropna=False here too

In [103]:
agg1 = (
    stop_times_with_trip.groupby(main_cols + ["stop_primary_direction"], dropna=False)
    .agg({"stop_sequence": "count"})
    .reset_index()
    .rename(columns={"stop_sequence": "total_stops"})
)

In [104]:
preview(agg1)

,schedule_gtfs_dataset_key,route_id,direction_id
0,73105f2d1cabc8170ab066d96863c5d5,1,NaN
4,73105f2d1cabc8170ab066d96863c5d5,11,NaN
8,73105f2d1cabc8170ab066d96863c5d5,12X,NaN
12,73105f2d1cabc8170ab066d96863c5d5,13X,NaN
16,73105f2d1cabc8170ab066d96863c5d5,1B,NaN
20,73105f2d1cabc8170ab066d96863c5d5,2,NaN
24,73105f2d1cabc8170ab066d96863c5d5,20,NaN
28,73105f2d1cabc8170ab066d96863c5d5,3,NaN
32,73105f2d1cabc8170ab066d96863c5d5,30,NaN
36,73105f2d1cabc8170ab066d96863c5d5,4,NaN


##### Making sure `dropna=False` really is the reason

In [105]:
agg_wo_dropna = (
    stop_times_with_trip.groupby(
        main_cols + ["stop_primary_direction"],
    )
    .agg({"stop_sequence": "count"})
    .reset_index()
    .rename(columns={"stop_sequence": "total_stops"})
)

In [106]:
preview(agg_wo_dropna)

,schedule_gtfs_dataset_key,route_id,direction_id
0,73105f2d1cabc8170ab066d96863c5d5,5,0.00
4,f5a749dd65924e025b1293c58f95f8d6,CC,0.00
7,f5a749dd65924e025b1293c58f95f8d6,CC,1.00
10,f5a749dd65924e025b1293c58f95f8d6,SF,0.00
11,f5a749dd65924e025b1293c58f95f8d6,SF,1.00
12,f5a749dd65924e025b1293c58f95f8d6,Shuttle,0.00
14,f5a749dd65924e025b1293c58f95f8d6,Shuttle,1.00


In [107]:
agg2 = agg1.sort_values(
    by=main_cols + ["total_stops"],
    ascending=[True, True, True, False],
)

In [110]:
cardinal_dir_df = (
    agg2.drop_duplicates(subset=main_cols)
    .reset_index(drop=True)
    .drop(columns=["total_stops"])
    .rename(columns={"stop_primary_direction": "route_primary_direction"})
)

In [111]:
preview(cardinal_dir_df)

,schedule_gtfs_dataset_key,route_id,direction_id
0,73105f2d1cabc8170ab066d96863c5d5,1,NaN
1,73105f2d1cabc8170ab066d96863c5d5,11,NaN
2,73105f2d1cabc8170ab066d96863c5d5,12X,NaN
3,73105f2d1cabc8170ab066d96863c5d5,13X,NaN
4,73105f2d1cabc8170ab066d96863c5d5,1B,NaN
5,73105f2d1cabc8170ab066d96863c5d5,2,NaN
6,73105f2d1cabc8170ab066d96863c5d5,20,NaN
7,73105f2d1cabc8170ab066d96863c5d5,3,NaN
8,73105f2d1cabc8170ab066d96863c5d5,30,NaN
9,73105f2d1cabc8170ab066d96863c5d5,4,NaN


In [114]:
# Merge cardinal direction & typology work
route_dir_metrics2 = pd.merge(
    route_dir_metrics, route_typologies, on=route_group_merge_cols, how="left"
).merge(cardinal_dir_df, on=route_group_merge_cols, how="left")

In [115]:
preview(route_dir_metrics2)

,schedule_gtfs_dataset_key,route_id,direction_id
0,73105f2d1cabc8170ab066d96863c5d5,1,NaN
1,73105f2d1cabc8170ab066d96863c5d5,11,NaN
2,73105f2d1cabc8170ab066d96863c5d5,12X,NaN
3,73105f2d1cabc8170ab066d96863c5d5,13X,NaN
4,73105f2d1cabc8170ab066d96863c5d5,1B,NaN
5,73105f2d1cabc8170ab066d96863c5d5,2,NaN
6,73105f2d1cabc8170ab066d96863c5d5,20,NaN
7,73105f2d1cabc8170ab066d96863c5d5,3,NaN
8,73105f2d1cabc8170ab066d96863c5d5,30,NaN
9,73105f2d1cabc8170ab066d96863c5d5,4,NaN


#### `gtfs_digest/merge_data` line 300 `df_avg_speeds` is also missing a lot of routes.
* HELP: not sure which file creates this ? 

In [116]:
SEGMENT_GCS

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/'

In [117]:
GTFS_DATA_DICT.rt_stop_times.route_dir_single_summary

'rollup_singleday/speeds_route_dir'

In [118]:
df_avg_speeds = merge_data.concatenate_speeds_by_route_direction(analysis_date_list)

In [119]:
df_avg_speeds2 = df_avg_speeds.loc[
    df_avg_speeds.schedule_gtfs_dataset_key.isin(schd_keys)
]

In [120]:
df_avg_speeds2.route_id.value_counts()

CC                                      43
5                                       27
b3848f93-d26b-48a9-b6a6-5de22a4eab47     9
Name: route_id, dtype: int64

In [125]:
df_avg_speeds2.head(2)

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,speed_mph,service_date
52005,73105f2d1cabc8170ab066d96863c5d5,5,0.00,all_day,15.48,2024-04-17
52006,73105f2d1cabc8170ab066d96863c5d5,5,0.00,all_day,16.89,2024-05-22


##### Side note, there are actually all of the Dec values for all time_periods here for speed.

In [128]:
df_avg_speeds2.loc[df_avg_speeds2.route_id == "5"]

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,speed_mph,service_date
52005,73105f2d1cabc8170ab066d96863c5d5,5,0.00,all_day,15.48,2024-04-17
52006,73105f2d1cabc8170ab066d96863c5d5,5,0.00,all_day,16.89,2024-05-22
52007,73105f2d1cabc8170ab066d96863c5d5,5,0.00,all_day,17.13,2024-06-12
52008,73105f2d1cabc8170ab066d96863c5d5,5,0.00,all_day,15.09,2024-07-17
52009,73105f2d1cabc8170ab066d96863c5d5,5,0.00,all_day,13.41,2024-08-14
52010,73105f2d1cabc8170ab066d96863c5d5,5,0.00,all_day,15.55,2024-09-18
52011,73105f2d1cabc8170ab066d96863c5d5,5,0.00,all_day,15.27,2024-10-16
52012,73105f2d1cabc8170ab066d96863c5d5,5,0.00,all_day,15.73,2024-11-13
52013,73105f2d1cabc8170ab066d96863c5d5,5,0.00,all_day,11.57,2024-12-11
52014,73105f2d1cabc8170ab066d96863c5d5,5,0.00,offpeak,13.36,2024-04-17


#### Dataframe in line 307 `df_rt_sched` in `gtfs_digest/merge_data`

In [130]:
df_rt_sched = merge_data.concatenate_rt_vs_schedule_by_route_direction(
    analysis_date_list
).astype({"direction_id": "float"})

In [131]:
df_rt_sched2 = df_rt_sched.loc[df_rt_sched.schedule_gtfs_dataset_key.isin(schd_keys)]

In [132]:
preview(df_rt_sched2)

,schedule_gtfs_dataset_key,route_id,direction_id
41032,73105f2d1cabc8170ab066d96863c5d5,5,0.00
41059,73105f2d1cabc8170ab066d96863c5d5,b3848f93-d26b-48a9-b6a6-5de22a4eab47,0.00
102289,f5a749dd65924e025b1293c58f95f8d6,CC,0.00
102325,f5a749dd65924e025b1293c58f95f8d6,CC,1.00


In [134]:
df_rt_sched2.head()

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,minutes_atleast1_vp,minutes_atleast2_vp,total_rt_service_minutes,total_scheduled_service_minutes,total_vp,vp_in_shape,is_early,is_ontime,is_late,n_vp_trips,vp_per_minute,pct_in_shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,pct_sched_journey_atleast1_vp,pct_sched_journey_atleast2_vp,rt_sched_journey_ratio,avg_rt_service_minutes,name,service_date,schedule_source_record_id
41032,73105f2d1cabc8170ab066d96863c5d5,5,0.00,all_day,2278,2267,2746.35,714.00,6801,2812,2,6,9,17,2.48,0.41,0.83,0.82,1.00,1.00,3.85,161.55,Santa Maria Schedule,2024-04-17,NaN
41033,73105f2d1cabc8170ab066d96863c5d5,5,0.00,all_day,1218,1201,1197.93,756.00,3609,2223,0,12,6,18,3.01,0.62,1.00,1.00,1.00,1.00,1.58,66.55,Santa Maria Schedule,2024-05-22,NaN
41034,73105f2d1cabc8170ab066d96863c5d5,5,0.00,all_day,1198,1188,1604.10,756.00,3559,2163,1,11,6,18,2.22,0.61,0.75,0.74,1.00,1.00,2.12,89.12,Santa Maria Schedule,2024-06-12,NaN
41035,73105f2d1cabc8170ab066d96863c5d5,5,0.00,all_day,1205,1189,1185.66,714.00,3573,2218,1,11,5,17,3.01,0.62,1.00,1.00,1.00,1.00,1.66,69.74,Santa Maria Schedule,2024-07-17,recxPy2JOcDFGDo31
41036,73105f2d1cabc8170ab066d96863c5d5,5,0.00,all_day,1135,1123,1116.70,630.00,3365,2967,3,9,3,15,3.01,0.88,1.00,1.00,1.00,1.00,1.77,74.45,Santa Maria Schedule,2024-08-14,recxPy2JOcDFGDo31


##### Go into file `rt_segment_speeds/segment_speed_utils/metrics.py`
* I think this file is used [here](https://github.com/cal-itp/data-analyses/blob/main/rt_scheduled_v_ran/scripts/rt_v_scheduled_routes.py) -> `rt_scheduled_v_ran/scripts/rt_v_scheduled_routes`

In [135]:
[*GTFS_DATA_DICT["stop_segments"]["route_dir_cols"]]

['route_id', 'direction_id']

In [136]:
dict_inputs = GTFS_DATA_DICT.rt_vs_schedule_tables

In [144]:
TRIP_EXPORT = dict_inputs.vp_trip_metrics

In [145]:
TRIP_EXPORT

'vp_trip/trip_metrics'

In [141]:
GTFS_DATA_DICT.rt_vs_schedule_tables.vp_route_direction_metrics

'vp_route_dir/route_direction_metrics'

In [140]:
dict_inputs.vp_route_direction_metrics

'vp_route_dir/route_direction_metrics'

In [142]:
ROUTE_EXPORT = dict_inputs.vp_route_direction_metrics

In [147]:
trip_df = pd.read_parquet(f"{RT_SCHED_GCS}{TRIP_EXPORT}_{one_analysis_date}.parquet")

In [148]:
trip_df.head(2)

,schedule_gtfs_dataset_key,trip_instance_key,route_id,direction_id,scheduled_service_minutes,total_vp,rt_service_minutes,minutes_atleast1_vp,minutes_atleast2_vp,vp_in_shape,route_short_name,sched_rt_category,time_of_day,peak_offpeak,vp_per_minute,pct_in_shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,pct_sched_journey_atleast1_vp,pct_sched_journey_atleast2_vp,rt_sched_journey_difference,is_early,is_ontime,is_late
0,3c62ad6ee589d56eca915ce291a5df0a,0001aafdcc1443da02a93a3596e67f36,41,0.00,50.00,165,54.52,56,55,156,41,schedule_and_vp,Early AM,offpeak,3.03,0.94,1.00,1.00,1.00,1.00,4.52,0,1,0
1,1770249a5a2e770ca90628434d4934b1,0001e1cccb0466096f3faed959b2f27b,4133,0.00,107.00,323,107.98,109,107,323,77,schedule_and_vp,Midday,offpeak,2.99,1.00,1.00,0.99,1.00,1.00,0.98,0,1,0


In [149]:
trip_df2 = trip_df.loc[trip_df.schedule_gtfs_dataset_key.isin(schd_keys)]

In [150]:
preview(trip_df2)

,schedule_gtfs_dataset_key,route_id,direction_id
127,73105f2d1cabc8170ab066d96863c5d5,30,NaN
407,73105f2d1cabc8170ab066d96863c5d5,3,NaN
1797,73105f2d1cabc8170ab066d96863c5d5,20,NaN
2622,73105f2d1cabc8170ab066d96863c5d5,4,NaN
3009,73105f2d1cabc8170ab066d96863c5d5,5,0.00
3096,73105f2d1cabc8170ab066d96863c5d5,11,NaN
3338,73105f2d1cabc8170ab066d96863c5d5,7,NaN
3512,73105f2d1cabc8170ab066d96863c5d5,9,NaN
4197,73105f2d1cabc8170ab066d96863c5d5,1,NaN
6708,73105f2d1cabc8170ab066d96863c5d5,12X,NaN


In [154]:
import sys

sys.path.append("../rt_scheduled_v_ran/scripts")
import rt_v_scheduled_routes

In [157]:
ROUTE_DIR_COLS = [*GTFS_DATA_DICT["stop_segments"]["route_dir_cols"]]

##### Somewhere in `rt_scheduled_v_ran/rt_v_scheduled_routes`, routes go missing.

In [158]:
crosswalk_cols = [
    "schedule_gtfs_dataset_key",
    "name",
    "schedule_source_record_id",
    "base64_url",
    "organization_source_record_id",
    "organization_name",
    "caltrans_district",
]

route_df = (
    metrics.concatenate_peak_offpeak_allday_averages(
        trip_df2,
        group_cols=["schedule_gtfs_dataset_key"] + ROUTE_DIR_COLS,
        metric_type="rt_vs_schedule",
    )
    .pipe(metrics.derive_rt_vs_schedule_metrics)
    .pipe(rt_v_scheduled_routes.average_rt_trip_times)
)

In [163]:
preview(route_df)

,schedule_gtfs_dataset_key,route_id,direction_id
0,73105f2d1cabc8170ab066d96863c5d5,5,0.00
2,f5a749dd65924e025b1293c58f95f8d6,CC,0.00
4,f5a749dd65924e025b1293c58f95f8d6,CC,1.00


In [160]:
route_df2 = metrics.concatenate_peak_offpeak_allday_averages(
    trip_df2,
    group_cols=["schedule_gtfs_dataset_key"] + ROUTE_DIR_COLS,
    metric_type="rt_vs_schedule",
)

In [162]:
preview(route_df2)

,schedule_gtfs_dataset_key,route_id,direction_id
0,73105f2d1cabc8170ab066d96863c5d5,5,0.00
2,f5a749dd65924e025b1293c58f95f8d6,CC,0.00
4,f5a749dd65924e025b1293c58f95f8d6,CC,1.00


##### `metrics.concatenate_peak_offpeak_allday_averages` references to `segment_calcs.calculate_avg_speeds` which lives in `rt_segment_speeds/segment_speed_utils`

In [165]:
["schedule_gtfs_dataset_key"] + ROUTE_DIR_COLS

['schedule_gtfs_dataset_key', 'route_id', 'direction_id']

### Fix Map: `gtfs_digest/merge_operator_data`

In [ ]:
OPERATOR_FILE = GTFS_DATA_DICT.digest_tables.operator_profiles
OPERATOR_ROUTE = GTFS_DATA_DICT.digest_tables.operator_routes_map

In [ ]:
operator_route_gdf = gpd.read_parquet(
    f"{RT_SCHED_GCS}{OPERATOR_ROUTE}.parquet",
)

In [ ]:
operator_route_gdf.columns

In [ ]:
operator_route_gdf2.columns

In [ ]:
len(operator_route_gdf2)

In [ ]:
operator_route_gdf2.is_rail.value_counts()

In [ ]:
operator_route_gdf2.organization_name.value_counts()

In [ ]:
operator_route_gdf2.schedule_gtfs_dataset_key.unique()

### Why does City of Santa Maria have multiple schedule_gtfs_dataset_keys?

In [ ]:
operator_route_gdf2.groupby(["organization_name", "schedule_gtfs_dataset_key"]).agg(
    {"route_short_name": "nunique"}
)

In [ ]:
operator_route_gdf2.drop(columns=["service_date"]).explore("organization_name")

In [ ]:
# operator_route_gdf2.drop(columns = ["service_date"]).explore("shape_array_key")

### Starting from here [`gtfs_funnel/operator_scheduled_stats`](https://github.com/cal-itp/data-analyses/blob/4dc340343a60b45ad94217c3efd91f807b03ebc2/gtfs_funnel/operator_scheduled_stats.py#L148)

In [ ]:
analysis_date = "2024-11-13"

In [ ]:
schd_keys = list(operator_route_gdf2.schedule_gtfs_dataset_key.unique())

#### Longest shape does have all the routes...

In [ ]:
longest_shape_gdf = operator_scheduled_stats.longest_shape_by_route(analysis_date)

In [ ]:
longest_shape_gdf2 = longest_shape_gdf.loc[
    longest_shape_gdf.schedule_gtfs_dataset_key.isin(schd_keys)
]

In [ ]:
longest_shape_gdf2.columns

In [ ]:
longest_shape_gdf2.info()

In [ ]:
longest_shape_gdf2.route_id.value_counts()

In [ ]:
# longest_shape_gdf2.explore("schedule_gtfs_dataset_key")

In [ ]:
longest_shape_gdf2.groupby(["schedule_gtfs_dataset_key", "route_id"]).agg(
    {"route_length_miles": "max"}
)

#### Somewhere along the way the routes are cut...maybe b/c of `direction_id`

In [ ]:
OPERATOR_EXPORT = GTFS_DATA_DICT.schedule_tables.operator_scheduled_stats

In [ ]:
SCHED_GCS

In [ ]:
GTFS_DATA_DICT.schedule_tables.operator_routes

In [ ]:
dec_url = "gs://calitp-analytics-data/data-analyses/gtfs_schedule/operator_profiles/operator_routes_2024-12-11.parquet"

In [ ]:
dec_df = gpd.read_parquet(dec_url)

In [ ]:
dec_df.organization_name.value_counts().head()

In [ ]:
dec_df.loc[
    dec_df.organization_name == "Alameda-Contra Costa Transit District"
].head().drop(columns=["geometry"]).T

In [ ]:
dec_df2 = dec_df.loc[dec_df.schedule_gtfs_dataset_key.isin(schd_keys)]

In [ ]:
dec_df2.shape

In [ ]:
type(dec_df2)

In [ ]:
dec_df2.drop(columns=["geometry"]).T

In [ ]:
# dec_df2.explore()

#### Find where in `gtfs_funnel` all the routes disappear

In [ ]:
group_cols = ["schedule_gtfs_dataset_key"]

In [ ]:
longest_shape_gdf2.info()

### something is going on in `operator_scheduled_stats.schedule_stats_by_operator`

In [ ]:
ROUTE_TYPOLOGY = GTFS_DATA_DICT.schedule_tables.route_typologies

In [ ]:
route_typology = pd.read_parquet(f"{SCHED_GCS}{ROUTE_TYPOLOGY}_{analysis_date}.parquet")

In [ ]:
from route_typologies import route_typologies

In [ ]:
route_typology_grouped = (
    route_typology.groupby(["schedule_gtfs_dataset_key", "route_id"])
    .agg({**{f"is_{c}": "sum" for c in route_typologies}})
    .reset_index()
)

In [ ]:
route_typology_grouped2 = route_typology_grouped.loc[
    route_typology_grouped.schedule_gtfs_dataset_key.isin(schd_keys)
]

#### Routes are missing for Santa Maria and Capitol Corridor in `ROUTE_TYPOLOGY`

In [ ]:
route_typology_grouped2.T

In [ ]:
route_gdf = longest_shape_gdf2.merge(
    route_typology_grouped2, on=["schedule_gtfs_dataset_key", "route_id"], how="outer"
)

In [ ]:
route_gdf.shape

In [ ]:
route_gdf.drop(columns=["geometry"])

In [ ]:
# route_gdf2.explore("schedule_gtfs_dataset_key")

### Change merge from `inner` to `left`

In [ ]:
f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.operator_routes_map}.parquet"

In [ ]:
SCHED_GCS

In [ ]:
GTFS_DATA_DICT.schedule_tables.operator_routes

In [ ]:
my_test_url = "gs://calitp-analytics-data/data-analyses/gtfs_schedule/operator_profiles/operator_routes_2024-12-11_AH.parquet"

In [ ]:
f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.operator_profiles}.parquet"

In [ ]:
test_gdf = gpd.read_parquet(my_test_url)

In [ ]:
test_gdf2 = test_gdf.loc[test_gdf.schedule_gtfs_dataset_key.isin(schd_keys)]

In [ ]:
test_gdf2.explore("route_id")

#### Test with all the dates.

In [ ]:
GTFS_DATA_DICT.schedule_tables.operator_routes

In [ ]:
RT_SCHED_GCS

In [ ]:
f"{OPERATOR_ROUTE}_AH_test"

In [ ]:
f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.operator_routes_map}.parquet"

In [ ]:
test_df = gpd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/rt_vs_schedule/digest/operator_routes_AH_test.parquet"
)

In [ ]:
test_df.columns

In [ ]:
op_routes_gdf = test_df.loc[test_df.organization_name.isin(org_name_lists)]

In [ ]:
# Find the most recent geography for each route.
op_routes_gdf = op_routes_gdf.sort_values(by=["service_date"], ascending=False)

# Keep only the most recent row.
op_routes_gdf = op_routes_gdf.drop_duplicates(
    subset=["route_long_name", "route_short_name", "route_combined_name"]
)

# Drop service_dates
op_routes_gdf = op_routes_gdf.drop(columns=["service_date"])

In [ ]:
op_routes_gdf.organization_name.value_counts()

In [ ]:
op_routes_gdf.loc[op_routes_gdf.organization_name == "City of Santa Maria"].explore(
    "route_long_name"
)